In [44]:
import pandas as pd
import tabula
import fitz
from markitdown import MarkItDown
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from openai import OpenAI
from dotenv import dotenv_values
import os
import re
openai_key = dotenv_values(".env")["OPENAI_KEY"]

In [55]:
doc = fitz.open('pura073552.ww.pdf')

page_range = range(35, 38)

for page_num in page_range:
	page = doc.load_page(page_num)
	
	new_page_pdf = fitz.open()
	new_page_pdf.insert_pdf(doc, from_page=page_num, to_page=page_num)
	page_path = f'pages/temp_{page_num}.pdf'
	new_page_pdf.save(page_path)
	new_page_pdf.close()

doc.close()

client = OpenAI(api_key=openai_key)
md = MarkItDown(llm_client=client, llm_model="gpt-4o")

files = sorted(list(map(lambda x: os.path.join("pages", x), os.listdir("pages"))))

tables_text = ""

for file in files:
	text_result = md.convert(file).text_content
	tables_text += text_result + "\n\n"

tables_text = tables_text.replace("Appendix 11. Characteristics of included studies\n\nTable A. Characteristics of included studies (n=126 studies; 338 datapoints)\n\n", "")

/Users/francip/Desktop/Master/PRVD/PRVD-1/env/lib/python3.13/site-packages/pdfminer/layout.py:361: ResourceWarning:

unclosed <ssl.SSLSocket fd=75, family=2, type=1, proto=0, laddr=('192.168.1.16', 60073), raddr=('162.159.140.245', 443)>



In [56]:
table_format = """
Author and Year,Study Design,Study Period,Country,Equity,Participants,Mean Age [Range],Risk of Bias,Number of dp,MA?,Exposure,Exposure Measure,Outcome Measure,N
Anastario 2020,Cross-sectional,NR,USA,High income country,"Youth attending 5 schools located on or near a tribal reservation in Montana",15.7 [14-18],Mod,2,Yes,"Freq. of SM use","Freq. of using Twitter to talk or learn about sex or any topic related to sex","No use of a condom at last sexual encounter",146
,,NR,USA,High income country,,15.7 [14-18],Mod,2,Yes,"Freq. of SM use","Freq. of using Facebook to talk or learn about sex or any topic related to sex","No use of a condom at last sexual encounter",146
Baker 2016,Cross-sectional,2009,USA,High income country,"Grade 6-12 urban school district students part of a federally funded project on school related initiatives",NR,High,3,Yes,"Freq. of SM use","Freq. of SNS use","Soft drug use (smoking, marijuana, alcohol) in the past month",3195
,,,,,,"Freq. of SNS use","Hard drug use (lifetime and past year)",3195
,,,,,,"Freq. of SNS use","Weapon carrying in the past month",3195

"""

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""Format the following text into csv using this structure: {table_format}. {tables_text}. Include all the data in the text, if any row have same data as above, please complete with that information.
            <csv>
            </csv>""",
        }
    ],
    stream=True,
)

final_result = ""
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        final_result += chunk.choices[0].delta.content
        print(chunk.choices[0].delta.content, end="")

Here's the text formatted into CSV according to the provided structure. I've included all rows from the provided data along with any that had repeated entries filled in with their information where applicable.

```csv
Author and Year,Study Design,Study Period,Country,Equity,Participants,Mean Age [Range],Risk of Bias,Number of dp,MA?,Exposure,Exposure Measure,Outcome Measure,N
Anastario 2020,Cross-sectional,NR,USA,High income country,"Youth attending 5 schools located on or near a tribal reservation in Montana","15.7 [14-18]",Mod,2,Yes,"Freq. of SM use","Freq. of using Twitter to talk or learn about sex or any topic related to sex","No use of a condom at last sexual encounter",146
Anastario 2020,Cross-sectional,NR,USA,High income country,,"15.7 [14-18]",Mod,2,Yes,"Freq. of SM use","Freq. of using Facebook to talk or learn about sex or any topic related to sex","No use of a condom at last sexual encounter",146
Baker 2016,Cross-sectional,2009,USA,High income country,"Grade 6-12 urban scho

In [57]:
csv = re.search(r"```csv(.*)```", final_result, re.DOTALL).group(1)
csv = csv.strip()

with open("table.csv", "w") as f:
	f.write(csv)


In [60]:
print(csv)

Author and Year,Study Design,Study Period,Country,Equity,Participants,Mean Age [Range],Risk of Bias,Number of dp,MA?,Exposure,Exposure Measure,Outcome Measure,N
Anastario 2020,Cross-sectional,NR,USA,High income country,"Youth attending 5 schools located on or near a tribal reservation in Montana","15.7 [14-18]",Mod,2,Yes,"Freq. of SM use","Freq. of using Twitter to talk or learn about sex or any topic related to sex","No use of a condom at last sexual encounter",146
Anastario 2020,Cross-sectional,NR,USA,High income country,,"15.7 [14-18]",Mod,2,Yes,"Freq. of SM use","Freq. of using Facebook to talk or learn about sex or any topic related to sex","No use of a condom at last sexual encounter",146
Baker 2016,Cross-sectional,2009,USA,High income country,"Grade 6-12 urban school district students part of a federally funded project on school related initiatives",NR,High,3,Yes,"Freq. of SM use","Freq. of SNS use","Soft drug use (smoking, marijuana, alcohol) in the past month",3195
Baker 2016,

In [59]:
df = pd.read_csv("table.csv", sep=",")

ParserError: Error tokenizing data. C error: Expected 14 fields in line 19, saw 15


In [4]:
page_range = range(35, 60)

for page_num in page_range:
	page = doc.load_page(page_num)
	
	new_page_pdf = fitz.open()
	new_page_pdf.insert_pdf(doc, from_page=page_num, to_page=page_num)
	page_path = f'pages/temp_{page_num}.pdf'
	new_page_pdf.save(page_path)

	table = tabula.read_pdf(page_path, pages=1)
	# md_result = md.result(page_path)
	# text_content = md_result.text_content
	break

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [5]:
data_page_35 = {
    "Author and year": ["Anastario 2020", "Baker 2016", "Baldwin 2018"],
    "Study design": ["Cross-sectional", "Cross-sectional", "Cross-sectional"],
    "Study period": ["NR", "2009", "2014"],
    "Country": ["USA", "USA", "Australia"],
    "Equity": ["High income country", "High income country", "High income country with mixed SEP"],
    "Participants": [
        "Youth attending 5 schools located on or near a tribal reservation in Montana",
        "Grade 6-12 urban school district students part of a federally funded project on school related initiatives",
        "Adolescents residing in New South Wales"
    ],
    "Mean age [range]": ["15.7 [14-18]", "NR", "NR [10-16]"],
    "Risk of bias": ["Mod", "High", "Low"],
    "N°. of dp": [2, 3, 7],
    "MA?": ["Yes", "Yes", "Yes"],
    "Exposure": [
        "Freq. of SM use",
        "Freq. of SM use",
        "Exposure to health-risk behaviour content"
    ],
    "Exposure measure": [
        "Freq. of using Twitter to talk or learn about sex or any topic related to sex",
        "Freq. of SNS use",
        "Watched food/beverage brand YouTube videos"
    ],
    "Outcome measure": [
        "No use of a condom at last sexual encounter",
        "Soft drug use (smoking, marijuana, alcohol) in the past month",
        "Freq. of unhealthy food consumption"
    ],
    "N": [146, 3195, 417]
}

df_page_35 = pd.DataFrame(data_page_35)

In [6]:
data_page_36 = [
    {
        "Author and year": "Ball 2020",
        "Study design": "Repeat cross-sectional",
        "Study period": "2016 & 2018",
        "Country": "New Zealand",
        "Equity": "High income country with mixed SEP",
        "Participants": "Year 10 students’ part of the Youth Insights Survey",
        "Mean age [range]": "NR [14-15]",
        "Risk of bias": "Low",
        "N° of dp": 2,
        "MA?": "Yes",
        "Exposure": "Freq. of SM use",
        "Exposure measure": "Freq. of using SM (status updates, uploading photos or videos) in the past week",
        "Outcome measure": "Current smoking (defined as smoking at least monthly)",
        "N": 5127
    },
    {
        "Author and year": "Ball 2020",
        "Study design": "Repeat cross-sectional",
        "Study period": "2016 & 2018",
        "Country": "New Zealand",
        "Equity": "High income country with mixed SEP",
        "Participants": "Year 10 students’ part of the Youth Insights Survey",
        "Mean age [range]": "NR [14-15]",
        "Risk of bias": "Low",
        "N° of dp": 2,
        "MA?": "Yes",
        "Exposure": "Freq. of online gambling",
        "Exposure measure": "Freq. of online gambling in the past week",
        "Outcome measure": "Current smoking (defined as smoking at least monthly)",
        "N": 5127
    },
    {
        "Author and year": "Baru 2020",
        "Study design": "Cross-sectional",
        "Study period": "2019",
        "Country": "Ethiopia",
        "Equity": "Low-middle income country with mixed SEP",
        "Participants": "Sexually active unmarried young female internal migrants residing in Barayu Town",
        "Mean age [range]": "18.9 [15-24]",
        "Risk of bias": "High",
        "N° of dp": 1,
        "MA?": "Yes",
        "Exposure": "Freq. of SM use",
        "Exposure measure": "Freq. of SM use",
        "Outcome measure": "Risk sexual behaviour (incl. multiple sexual partners; sex without condoms or inconsistent condom use; initiation of sex before the age of 18 years; sexual intercourse under the influence of substances)",
        "N": 150
    },
    {
        "Author and year": "Baumgartner 2012",
        "Study design": "Cohort",
        "Study period": "2018",
        "Country": "Netherlands",
        "Equity": "High income country",
        "Participants": "Adolescents",
        "Mean age [range]": "14.5 [12-18]",
        "Risk of bias": "Mod",
        "N° of dp": 1,
        "MA?": "Yes",
        "Exposure": "Freq. of SM use",
        "Exposure measure": "Freq. of online communication",
        "Outcome measure": "Online sexual risk behaviours (incl. searching for someone on the internet to have sex with; sending a photo or video in which they were partly naked to someone they knew only online)",
        "N": 1345
    },
    {
        "Author and year": "Bayraktar 2007",
        "Study design": "Cross-sectional",
        "Study period": "NR",
        "Country": "Cyprus",
        "Equity": "HIC",
        "Participants": "Elementary and high school students residing in North Cyprus",
        "Mean age [range]": "14.4 [NR]",
        "Risk of bias": "High",
        "N° of dp": 1,
        "MA?": "No",
        "Exposure": "Exposure to health-risk behaviour content",
        "Exposure measure": "Online gaming: fighting games",
        "Outcome measure": "Anti-social aggression",
        "N": 686
    },
    {
        "Author and year": "Beebe 2004",
        "Study design": "Cross-sectional",
        "Study period": "2001",
        "Country": "USA",
        "Equity": "High income country",
        "Participants": "Grade 9 school students’ part of the Minnesota Student Survey",
        "Mean age [range]": "14.7 [13-17]",
        "Risk of bias": "High",
        "N° of dp": 12,
        "MA?": "Yes",
        "Exposure": "Freq. of SM use",
        "Exposure measure": "Presence of internet chat room use",
        "Outcome measure": "Tobacco use in the past year",
        "N": 40376
    },
    {
        "Author and year": "Boers 2020",
        "Study design": "Cohort",
        "Study period": "NR",
        "Country": "Canada",
        "Equity": "High income country with low SEP",
        "Participants": "Grade 7 school students, part of the Co-Venture Preventure study",
        "Mean age [range]": "12.7 [NR]",
        "Risk of bias": "Low",
        "N° of dp": 1,
        "MA?": "Yes",
        "Exposure": "Time spent on SM",
        "Exposure measure": "Time spent on SM per day",
        "Outcome measure": "Freq. of alcohol consumption",
        "N": 3612
    }
]

# Convert to DataFrame for better readability
df_page_36 = pd.DataFrame(data_page_36)

In [7]:
data_page_37 = {
    "Author and year": [
        "Boniel-Nissim 2022",
        "Booker 2015",
        "Brunborg 2019",
        "Brunborg 2022"
    ],
    "Study design": [
        "Cross-sectional",
        "Cross-sectional",
        "Cohort",
        "Cohort"
    ],
    "Study period": [
        "2017-2018",
        "2009",
        "2014-2015",
        "2017-2020"
    ],
    "Country": [
        "42 countries and regions across Europe, North America, and the Middle East",
        "UK",
        "Norway",
        "Norway"
    ],
    "Equity": [
        "High income country with mixed SEP",
        "High income country with mixed SEP",
        "High income country",
        "High income country with mixed SEP"
    ],
    "Participants": [
        "School students, part of the Health Behaviour in School-aged Children Survey",
        "Sample members of the youth panel of The UK Household Longitudinal Study",
        "Grade 8-10 and 1-2nd year high school students’ part of the pilot Monitoring Young Lifestyles Project",
        "Middle school adolescents’ part of the MyLife Study"
    ],
    "Mean age [range]": [
        "13.6 [11-15]",
        "NR [10-15]",
        "15.2 [13-17]",
        "14.3 [12.8-16.8]"
    ],
    "Risk of bias": [
        "Low",
        "High",
        "Low",
        "Mod"
    ],
    "N°. of dp": [4, 1, 4, 1],
    "MA?": ["Yes", "No", "No", "No"],
    "Exposure": [
        "Freq. of SM use",
        "Time spent on SM",
        "Time spent on SM",
        "Time spent on SM"
    ],
    "Exposure measure": [
        "Freq. of online contact with others via SM (via validated tool)",
        "Time spent chatting on social websites on a normal school day",
        "Change in hrs of SM use per day (Δ=T2–T1)",
        "Average number of hrs spent on SM per day"
    ],
    "Outcome measure": [
        "Smoking (≥1 × in the last month) (via validated tool)",
        "Sports participation",
        "Change in episodic heavy drinking freq.",
        "Change in alcohol use (via AUDIT-C)"
    ],
    "N": [173577, 4899, 763, 3096]
}

df_page_37 = pd.DataFrame(data_page_37)

In [8]:
data_page_38 = {
    "Author and year": [
        "Camenga 2018",
        "Canale 2016",
        "Casaló 2022",
        "Cavazos-Rehg 2014"
    ],
    "Study design": [
        "Cohort",
        "Cross-sectional",
        "Cross-sectional",
        "Cross-sectional"
    ],
    "Study period": [
        "2013-2014",
        "2013",
        "2016-2017",
        "2011"
    ],
    "Country": [
        "USA",
        "Italy",
        "Spain",
        "USA"
    ],
    "Equity": [
        "High income country with mixed SEP",
        "High income country with mixed SEP",
        "High income country with mixed SEP",
        "High income country with mixed SEP"
    ],
    "Participants": [
        "High and middle school students’ part of a longitudinal school-based cohort study",
        "High school students’ part of the European School Survey Project on Alcohol and Other Drugs Italia",
        "Secondary education students, part of the National Survey on Drug Use Among High School Students in Spain",
        "Grade 6-12 school students’ part of the National Youth Tobacco Survey"
    ],
    "Mean age [range]": [
        "14.1 [NR]",
        "17.2 [15-19]",
        "NR [14-18]",
        "NR [11-17]"
    ],
    "Risk of bias": [
        "High",
        "Low",
        "Low",
        "High"
    ],
    "N°. of dp": [4, 3, 4, 1],
    "MA?": ["Yes", "Yes", "Yes", "Yes"],
    "Exposure": [
        "Exposure to health-risk behaviour content",
        "Freq. of SM use",
        "Time spent on SM",
        "Exposure to health-risk behaviour content"
    ],
    "Exposure measure": [
        "Exposure to e-cigarette advertisements on Facebook",
        "Freq. of using internet for leisure activities (e.g., online chatting)",
        "Time spent on SNS per day",
        "Exposure to tobacco ads/promotions via Facebook/Myspace in the past month"
    ],
    "Outcome measure": [
        "Ever e-cigarette use",
        "Problem gambling (via SOGS-RA)",
        "Sports frequency 1-3 days per year",
        "Used any form of tobacco in the past month"
    ],
    "N": [1742, 14478, 35369, 15673]
}

df_page_38 = pd.DataFrame(data_page_38)

df = pd.concat([df_page_35, df_page_36, df_page_37, df_page_38], ignore_index=True)
df.to_csv('data.csv', index=False)

In [9]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Reporte de Estudios sobre Redes Sociales y Conductas Juveniles"),
    
    html.Div([
        html.Label("Seleccione un indicador para visualizar:"),
        dcc.Dropdown(
            id="indicator-dropdown",
            options=[
                {"label": "Número de Participantes", "value": "N"},
                {"label": "Edad Media", "value": "Mean age [range]"},
            ],
            value="N",
        ),
    ], style={"margin-bottom": "20px"}),
    
    dcc.Graph(id="bar-chart"),
    
    html.Div([
        html.H3("Tabla Resumen de los Datos"),
        dcc.Graph(
            id="table",
            figure=px.bar(
                df,
                x="Author and year",
                y="N",
                text="Outcome measure",
                title="Participantes por Estudio",
                labels={"N": "Número de Participantes", "Author and year": "Autor y Año"},
            ).update_traces(marker_color="skyblue"),
        ),
    ]),
])

@app.callback(
    Output("bar-chart", "figure"),
    [Input("indicator-dropdown", "value")],
)
def update_chart(indicator):
    if indicator == "N":
        title = "Número de Participantes por Estudio"
        y_axis_label = "Número de Participantes"
    else:
        title = "Edad Media por Estudio"
        y_axis_label = "Edad Media"
    
    fig = px.bar(
        df,
        x="Author and year",
        y=indicator,
        title=title,
        labels={indicator: y_axis_label, "Author and year": "Autor y Año"},
        text="Outcome measure",
    )
    fig.update_traces(marker_color="orange")
    return fig

# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server(debug=True)

/Users/francip/Desktop/Master/PRVD/PRVD-1/env/lib/python3.13/site-packages/dash/dash.py:2282: DeprecationWarning:

Dash.run_server is deprecated and will be removed in Dash 3.0

/Users/francip/Desktop/Master/PRVD/PRVD-1/env/lib/python3.13/site-packages/dash/dash.py:1814: DeprecationWarning:

'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead

